In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain_qdrant import Qdrant

huggingFaceApiToken = os.environ["HUGGINGFACEHUB_API_TOKEN"]
qdrant_url = os.environ["QDRANT_URL"]
# embedding
embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=huggingFaceApiToken, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

# LLM
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    max_length=128,
    temperature=0.5,
    huggingfacehub_api_token=huggingFaceApiToken,
)

# Retriever
store = Qdrant.from_existing_collection(
    embedding=embedding,
    collection_name="lilianweng_agent",
    url=qdrant_url,
)
retriever = store.as_retriever(search_kwargs={"k": 1})


def printEverything(everything, name=""):
    print(f"{name} {everything}")
    return everything

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\HA HA\.cache\huggingface\token
Login successful


In [3]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser() 
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

'\n\nTitle: Advancements in Large-scale Language Modeling Agents: A Focus on Task Decomposition\n\nAbstract:\nThis paper explores the concept of task decomposition in the context of large-scale language modeling agents (LLM agents). The ability to decompose complex tasks into smaller, manageable sub-tasks is a crucial aspect of artificial intelligence, particularly in the realm of LLM agents, which are designed to process vast amounts of data and generate human-like responses.\n\nIntroduction:\nLarge-scale Language Modeling Agents (LLM agents) have made significant strides in recent years, demonstrating remarkable capabilities in understanding and generating human-like text. However, as these agents continue to grow in complexity, the challenge of managing and processing the vast amounts of data they handle becomes increasingly daunting. To address this issue, researchers have turned to task decomposition, a methodology that allows complex tasks to be broken down into smaller, more man

In [4]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(page_content='Finite context length: The restricted context capacity limits the inclusion of historical information, detailed instructions, API call context, and responses. The design of the system has to work with this limited communication bandwidth, while mechanisms like self-reflection to learn from past mistakes would benefit a lot from long or infinite context windows. Although vector stores and retrieval can provide access to a larger knowledge pool, their representation power is not as powerful as full attention.\n\n\nChallenges in long-term planning and task decomposition: Planning over a lengthy history and effectively exploring the solution space remain challenging. LLMs struggle to adjust plans when faced with unexpected errors, making them less robust compared to humans who learn from trial and error.\n\n\nReliability of natural language interface: Current agent system relies on natural language as an interface between LLMs and external components such as memory 

In [5]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'\nAnswer: Task decomposition for LLM (Large Language Model) agents refers to the process of breaking down complex tasks into smaller, manageable sub-tasks. This is important because LLMs may struggle with planning over a lengthy history and adjusting plans when faced with unexpected errors. Effective task decomposition can help make the agents more robust and efficient in their problem-solving abilities. However, as mentioned in the context, LLMs can have challenges with this due to their limited context capacity and the reliability issues with their natural language interface.'